# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


In [2]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 6.49MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [3]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain_chroma-0.1.4-py3-none-any.whl.metadata (1.6 kB)
  Using cached Chroma-0.2.0.tar.gz (5.8 kB)
  Preparing metadata (setup.py) ... done
  Using cached langchain_groq-0.2.0-py3-none-any.whl.metadata (2.9 kB)
ERROR: Could not find a version that satisfies the requirement LLMChain (from versions: none)
ERROR: No matching distribution found for LLMChain


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [4]:
import pandas as pd

df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [5]:
!pip install langchain_community

  Using cached langchain_community-0.3.0-py3-none-any.whl.metadata (2.8 kB)
  Using cached langchain-0.3.0-py3-none-any.whl.metadata (7.1 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 426.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.1/405.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.0/290.0 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: te

In [6]:
from langchain_community.document_loaders.csv_loader import CSVLoader

In [7]:
loader = CSVLoader(file_path='/content/Dataset.csv')
data = loader.load()

In [8]:
len(data)

104

In [9]:
for record in data[:0]:
    print(record.page_content)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [11]:
RecursiveCharacterTextSplitter?

In [12]:
text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n"] ,

)

In [13]:
documents = text_splitter.split_documents(data)

In [14]:
len(documents)

104

In [15]:
documents[0].page_content

'\ufeffالمخالفة: قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.\nالغرامة: الغرامة المالية 100 - 150 ريال'

In [16]:
for i in range(5):
  print()
  print(documents[i].page_content)
  print()


﻿المخالفة: قيادة المركبة في الأسواق التي لا يسمح بالقيادة فيها.
الغرامة: الغرامة المالية 100 - 150 ريال


﻿المخالفة: ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.
الغرامة: الغرامة المالية 100 - 150 ريال


﻿المخالفة: عدم وجود تأمين ساري للمركبة.
الغرامة: الغرامة المالية 100 - 150 ريال


﻿المخالفة: عبور المشاة للطرق من غير الأماكن المخصصة لهم.
الغرامة: الغرامة المالية 100 - 150 ريال


﻿المخالفة: عدم تقيد المشاة بالإشارات الخاصة بهم.
الغرامة: الغرامة المالية 100 - 150 ريال



## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [17]:
!pip install langchain_huggingface

  Using cached langchain_huggingface-0.1.0-py3-none-any.whl.metadata (1.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 15.4 MB/s eta 0:00:00


In [18]:
%pip install -qU langchain-huggingface

In [19]:
!pip install -qU "langchain-chroma>=0.1.2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 111.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.7/149.7 kB 14.9 MB/s eta 0:00

In [20]:
from langchain_chroma import Chroma

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings

In [22]:
HuggingFaceEmbeddings?

In [23]:
!pip install -U sentence-transformers

In [24]:
EMBEDDING_MODEL = "acayir64/arabic-embedding-model-only-pairs"

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL,
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [26]:
persist_directory = "./chroma_db"
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory=persist_directory
)

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [66]:
from langchain.prompts import PromptTemplate

PROMPT_TEMPLATE="""
give advice in Arabic. if you do dont know an answer say لا املك خلفيه عن هذا الموضوع. الرجاء اسالني عن المخالفات المرورية, and do not make up answers. answer based only on the following context:
Context: {context}
Advice: {advice}
Your answer:
"""

prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "advice"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [28]:
!pip install -qU langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.4 MB/s eta 0:00:00


In [36]:
from langchain_groq import ChatGroq


groq_api_key = ''
llm = ChatGroq(
    model="llama3-8b-8192",
    temperature=0,
    groq_api_key = groq_api_key
)

## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [67]:
from langchain.chains import LLMChain

MODEL_combined = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [68]:
def query_rag(user_query):

    similarity_search_results = vectordb.similarity_search_with_score(user_query, k=3)
    context_text = "\n\n".join([document.page_content for document, _score in similarity_search_results])

    rag_response = MODEL_combined.invoke({"context": context_text, "advice": user_query})

    return rag_response

In [51]:
response = query_rag("هل هناك مخالفة علي ترك السياره مفتوحه")
response

Prompt after formatting:

give advice in Arabic. if you do dont know an answer say you do not have any background, and do not make up answers. answer based only on the following context:
Context: ﻿المخالفة: عدم التقيد بإشارات رجل الأمن اليدوية عند تنظيمه للحركة وعدم إعطاء إشارته الأولوية على الإشارات الضوئية.
الغرامة: الغرامة المالية 500 - 900 ريال

﻿المخالفة: عدم استخدام مقاعد الأمان المخصصة للأطفال.
الغرامة: الغرامة المالية 300 - 500 ريال

﻿المخالفة: عدم استخدام الأنوار اللازمة عند السير ليلاً أو في الأحوال الجوية التي تكون فيها الرؤية غير واضحة.
الغرامة: الغرامة المالية 1000 - 2000 ريال
Advice: هل هناك مخالفة علي ترك السياره مفتوحه
Your answer:


> Finished chain.


{'context': '\ufeffالمخالفة: عدم التقيد بإشارات رجل الأمن اليدوية عند تنظيمه للحركة وعدم إعطاء إشارته الأولوية على الإشارات الضوئية.\nالغرامة: الغرامة المالية 500 - 900 ريال\n\n\ufeffالمخالفة: عدم استخدام مقاعد الأمان المخصصة للأطفال.\nالغرامة: الغرامة المالية 300 - 500 ريال\n\n\ufeffالمخالفة: عدم استخدام الأنوار اللازمة عند السير ليلاً أو في الأحوال الجوية التي تكون فيها الرؤية غير واضحة.\nالغرامة: الغرامة المالية 1000 - 2000 ريال',
 'advice': 'هل هناك مخالفة علي ترك السياره مفتوحه',
 'text': 'I don\'t have any background in Arabic or traffic laws, so I won\'t provide an answer that might be incorrect. However, I can try to help you with a general response.\n\nAccording to the context you provided, it seems that there are specific traffic violations and their corresponding fines. To answer your question, I would suggest that you check the specific traffic laws and regulations in your area to see if leaving a car open is considered a violation.\n\nIf you\'re looking for a general advic

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [52]:
response = query_rag("ماهي الغرامة على القيادة بدون رخصة؟")
response

Prompt after formatting:

give advice in Arabic. if you do dont know an answer say you do not have any background, and do not make up answers. answer based only on the following context:
Context: ﻿المخالفة: قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: استخدام السائق بيده أي جهاز محمول أثناء سير المركبة.
الغرامة: الغرامة المالية 500 - 900 ريال
Advice: ماهي الغرامة على القيادة بدون رخصة؟
Your answer:


> Finished chain.


{'context': '\ufeffالمخالفة: قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.\nالغرامة: الغرامة المالية 1000 - 2000 ريال\n\n\ufeffالمخالفة: القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.\nالغرامة: الغرامة المالية 1000 - 2000 ريال\n\n\ufeffالمخالفة: استخدام السائق بيده أي جهاز محمول أثناء سير المركبة.\nالغرامة: الغرامة المالية 500 - 900 ريال',
 'advice': 'ماهي الغرامة على القيادة بدون رخصة؟',
 'text': 'According to the provided context, the fine for driving without a license is 1000-2000 Riyals. Therefore, my advice would be:\n\n"الغرامة على القيادة بدون رخصة هي 1000-2000 ريال."\n\nTranslation: "The fine for driving without a license is 1000-2000 Riyals."'}

In [53]:
print(f'Context:\n{response["context"]}\n\nQuestion:\n{response["advice"]}\n\nAdvice: \n{response["text"]}')

Context:
﻿المخالفة: قيادة المركبة قبل الحصول على رخصة قيادة أو في حال سحب الرخصة.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: القيادة على أكتاف الطريق وعلى الأرصفة أو المسارات التي يمنع القيادة فيها.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: استخدام السائق بيده أي جهاز محمول أثناء سير المركبة.
الغرامة: الغرامة المالية 500 - 900 ريال

Question:
ماهي الغرامة على القيادة بدون رخصة؟

Advice: 
According to the provided context, the fine for driving without a license is 1000-2000 Riyals. Therefore, my advice would be:

"الغرامة على القيادة بدون رخصة هي 1000-2000 ريال."

Translation: "The fine for driving without a license is 1000-2000 Riyals."


In [54]:
response2 = query_rag("مخالفات قطع الاشاره على ماذا تقتضي؟")
response2

Prompt after formatting:

give advice in Arabic. if you do dont know an answer say you do not have any background, and do not make up answers. answer based only on the following context:
Context: ﻿المخالفة: عدم تثبيت اللوحة في مكانها المخصص.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: عدم وجود إنارة جانبية أو عواكس أو سواتر للشاحنات والمقطورات.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: مخالفة قواعد استعمال أنوار التلاقي.
الغرامة: الغرامة المالية 150 - 300 ريال
Advice: مخالفات قطع الاشاره على ماذا تقتضي؟
Your answer:


> Finished chain.


{'context': '\ufeffالمخالفة: عدم تثبيت اللوحة في مكانها المخصص.\nالغرامة: الغرامة المالية 1000 - 2000 ريال\n\n\ufeffالمخالفة: عدم وجود إنارة جانبية أو عواكس أو سواتر للشاحنات والمقطورات.\nالغرامة: الغرامة المالية 1000 - 2000 ريال\n\n\ufeffالمخالفة: مخالفة قواعد استعمال أنوار التلاقي.\nالغرامة: الغرامة المالية 150 - 300 ريال',
 'advice': 'مخالفات قطع الاشاره على ماذا تقتضي؟',
 'text': "I can provide advice in Arabic based on the given context. Here's my answer:\n\nمخالفات قطع الإشارة على ماذا تقتضي؟\n\nأيها المخالف، إذا قطعك الإشارة، فمن الأفضل أن تتباطأ وتتوقف عن الحركة فوراً. هذا لأن الإشارة هي من أهم وسائل الإشارة المرورية، وتقوم بتحذيرك من وجود مخاطر أو مشكلات على الطريق. قطع الإشارة دون أن تتباطأ يمكن أن يسبب حوادث أو مشكلات خطيرة.\n\nلذلك، إذا قطعك الإشارة، فمن الأفضل أن تتباطأ وتتوقف عن الحركة فوراً، ثم تنتظر الإشارة مرة أخرى قبل أن تcontinue الحركة. هذا سوف يمنعك من وقوع أي مشكلات أو حوادث على الطريق.\n\nNote: I don't have any background in traffic laws or regulations, so I'm pr

In [55]:
print(f'Context:\n{response2["context"]}\n\nQuestion:\n{response2["advice"]}\n\nAdvice: \n{response2["text"]}')

Context:
﻿المخالفة: عدم تثبيت اللوحة في مكانها المخصص.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: عدم وجود إنارة جانبية أو عواكس أو سواتر للشاحنات والمقطورات.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: مخالفة قواعد استعمال أنوار التلاقي.
الغرامة: الغرامة المالية 150 - 300 ريال

Question:
مخالفات قطع الاشاره على ماذا تقتضي؟

Advice: 
I can provide advice in Arabic based on the given context. Here's my answer:

مخالفات قطع الإشارة على ماذا تقتضي؟

أيها المخالف، إذا قطعك الإشارة، فمن الأفضل أن تتباطأ وتتوقف عن الحركة فوراً. هذا لأن الإشارة هي من أهم وسائل الإشارة المرورية، وتقوم بتحذيرك من وجود مخاطر أو مشكلات على الطريق. قطع الإشارة دون أن تتباطأ يمكن أن يسبب حوادث أو مشكلات خطيرة.

لذلك، إذا قطعك الإشارة، فمن الأفضل أن تتباطأ وتتوقف عن الحركة فوراً، ثم تنتظر الإشارة مرة أخرى قبل أن تcontinue الحركة. هذا سوف يمنعك من وقوع أي مشكلات أو حوادث على الطريق.

Note: I don't have any background in traffic laws or regulations, so I'm providing general advice based on common 

In [69]:
response3 = query_rag("ما حال الطقس اليوم")
response3

Prompt after formatting:

give advice in Arabic. if you do dont know an answer say لا املك خلفيه عن هذا الموضوع. الرجاء اسالني عن المخالفات المرورية, and do not make up answers. answer based only on the following context:
Context: ﻿المخالفة: عدم استخدام الأنوار اللازمة عند السير ليلاً أو في الأحوال الجوية التي تكون فيها الرؤية غير واضحة.
الغرامة: الغرامة المالية 1000 - 2000 ريال

﻿المخالفة: قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.
الغرامة: الغرامة المالية 500 - 900 ريال

﻿المخالفة: تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر.
الغرامة: الغرامة المالية 3000 - 6000 ريال
Advice: ما حال الطقس اليوم
Your answer:


> Finished chain.


{'context': '\ufeffالمخالفة: عدم استخدام الأنوار اللازمة عند السير ليلاً أو في الأحوال الجوية التي تكون فيها الرؤية غير واضحة.\nالغرامة: الغرامة المالية 1000 - 2000 ريال\n\n\ufeffالمخالفة: قيادة المركبة داخل الأنفاق من غير إضاءة أنوارها.\nالغرامة: الغرامة المالية 500 - 900 ريال\n\n\ufeffالمخالفة: تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر.\nالغرامة: الغرامة المالية 3000 - 6000 ريال',
 'advice': 'ما حال الطقس اليوم',
 'text': 'ما حال الطقس اليوم؟\n\nلا املك خلفيه عن هذا الموضوع، ولكنني يمكن أن أقول لك أن من الأفضل أن تحرص على استخدام الأنوار اللازمة عند السير ليلاً أو في الأحوال الجوية التي تكون فيها الرؤية غير واضحة، لماذا؟ لأن عدم استخدام الأنوار اللازمة يمكن أن يسبب مخالفة مرورية، والتي قد تترتب عليها غرامة مالية.\n\nأيضاً، إذا كنت ستحتاج إلى معرفة حالة الطقس اليوم، يمكنك أن تطلع على الأخبار أو تتحقق من موقع الويب الرسمي للهيئة الوطنية للطقس.'}